In [ ]:
import random
import numpy as np
import pandas as pd

In [ ]:
# code sourced from https://medium.com/@anthonytapias/build-a-deck-of-cards-with-oo-python-c41913a744d3
class Card : 
  def __init__ (self, color, val, action) : # add action
    self.color = color
    self.val = val
    self.action = action

  def show(self):
    print(self.color, self.val, self.action)


In [ ]:
class Deck :
  def __init__(self, special_cards = True):
    self.cards = []
    self.build(special_cards)
  
  # method to construct deck
  def build(self, special_cards = True):
    for c in ["Red", "Blue", "Green", "Yellow"] :
      # number cards-- 2 per color 1-9, 1 per color for 0
      for num in range(0, 10) :
        self.cards.append(Card(c, num, None))
        if(num > 0) :
            self.cards.append(Card(c, num, None))
      if special_cards == True:
        # action cards-- 2 of each color
        for i in range(2):
          self.cards.append(Card(c, None, "Draw 2"))
          self.cards.append(Card(c, None, "Reverse"))
          self.cards.append(Card(c, None, "Skip"))
    if special_cards == True:
      # wild cards -- 4 of each type
      for i in range(4):
        self.cards.append(Card(None, None, "Wild Color"))
        self.cards.append(Card(None, None, "Wild Draw 4"))

  def show(self) : 
     for c in self.cards :
      c.show()
  
  def shuffle(self) :
    random.shuffle(self.cards)

  def drawCard(self) :
    return self.cards.pop()

In [ ]:
class Player :
  def __init__(self, name, strategy) :
    self.name = name
    self.strategy = strategy
    self.hand = []

  def draw(self, deck) :
    card = deck.drawCard()
    self.hand.append(card)
    return card

  def showHand(self) :
    for card in self.hand :
      card.show()
    print("\n")

  def get_color_preference(self):
    """method to get color preference when wild card played"""
    color_freq = {"Blue":0, "Red":0, "Yellow":0, "Green":0}
    # populate color_freq dictionary with color frequencies
    for card in self.hand:
      if card.color is not None:
        color_freq[card.color] += 1
    # list to include colors with highest frequency
    preferences = []
    # add colors with highest frequency to preferences list
    for key, value in color_freq.items():
       if value == max(color_freq.values()):
         preferences.append(key)

    # return random choice from preferences list
    choice = random.choice(preferences)

    return choice

In [ ]:
def dealCards(listOfPlayers, deck, numberOfCards = 7) :
  for i in range(numberOfCards) :
    for j in listOfPlayers : 
      j.draw(deck)

In [ ]:
# function to check for matches
def check_matches(target_card, hand):
  matches = [] # list of matches
  for c in hand: # if card in hand matches color or number of target card, add to list of matches
    if (c.color == target_card.color) or (c.val == target_card.val) or ((c.action == target_card.action) and (c.action is not None)):
      matches.append(c)
    # if wild card, it will always be a match
    if (c.action == "Wild Color") or (c.action == "Wild Draw4"):
      matches.append(c)
  return matches

In [ ]:
# function to redirect to different strategy functions for choosing card
def choose_card(target_card, player, deck):

  # get list of matches
  matches = check_matches(target_card, player.hand)
  # if no matches, draw a card, check if it is a match, play if it is
  if len(matches) == 0:
    draw_card = player.draw(deck)
    if (draw_card.color == target_card.color) or (draw_card.val == target_card.val):
      return draw_card
    else:
      return None
  # if there is only one match, play it
  if len(matches) == 1:
    return matches[0]

  # if there is more than one match, choose which one to play based on strategy
  if player.strategy == "random":
    card_to_play = choose_card_random(target_card, player, deck, matches)
  elif player.strategy == "color":
    card_to_play = choose_card_color(target_card, player, deck, matches)
  elif player.strategy == "number":
    card_to_play = choose_card_number(target_card, player, deck, matches)
  elif player.strategy == "save specials":
    card_to_play = choose_card_savespecials(target_card, player, deck, matches)
  elif player.strategy == "use specials":
    card_to_play = choose_card_usespecials(target_card, player, deck, matches)
  return card_to_play

In [ ]:
# function to choose card randomly
def choose_card_random(target_card, player, deck, matches):
  card_to_play = random.choice(matches)
  return card_to_play

In [ ]:
# function to choose card with preference of color over number
def choose_card_color(target_card, player, deck, matches):
  
  color_matches = []
  for c in matches:
    if c.color == target_card.color:
      color_matches.append(c)
  # if color matches exist, play one of those
  if len(color_matches) > 0:
    card_to_play = random.choice(color_matches)
    # print("preference applied")
  # if not, play a random card
  else:
    card_to_play = random.choice(matches)
    
  # returns card to play, or None if there are no matches
  return card_to_play


In [ ]:
# function to choose card with preference of number over color
def choose_card_number(target_card, player, deck, matches):
  
  number_matches = []
  for c in matches:
    if c.val == target_card.val:
      number_matches.append(c)
  # if color matches exist, play one of those
  if len(number_matches) > 0:
    card_to_play = random.choice(number_matches)
    # print("preference applied")
  # if not, play a random card
  else:
    card_to_play = random.choice(matches)
    
  # returns card to play, or None if there are no matches
  return card_to_play

In [ ]:
# function to choose card with preference for non special cards
def choose_card_savespecials(target_card, player, deck, matches):

  nonspecial_matches = []
  for c in matches:
    if c.action is None:
      nonspecial_matches.append(c)
  # if non-special matches exist, play one of those
  if len(nonspecial_matches) > 0:
    card_to_play = random.choice(nonspecial_matches)
    # print("preference applied")
  # if not, play a random card
  else:
    card_to_play = random.choice(matches)
    
  # returns card to play, or None if there are no matches
  return card_to_play

In [ ]:
# function to choose card with preference for special cards
def choose_card_usespecials(target_card, player, deck, matches):
  
  special_matches = []
  for c in matches:
    if c.action is not None:
      special_matches.append(c)
  # if special matches exist, play one of those
  if len(special_matches) > 0:
    card_to_play = random.choice(special_matches)
    #print("preference applied")
  # if not, play a random card
  else:
    card_to_play = random.choice(matches)
    
  # returns card to play, or None if there are no matches
  return card_to_play

In [ ]:
def play_game(numPlayers = 2, strategies = ("random", "random"), special_cards = True, seed = 1):
  discard_pile = []
  # create and shuffle deck
  random.seed(seed)
  deck = Deck(special_cards)
  deck.shuffle()

  # create players
  players = []
  for player in range(numPlayers) :
    temp_player = Player("p" + str(player), strategies[player])
    players.append(temp_player)
  
  # randomize order of players
  random.shuffle(players)

  # print("Players")
  # for p in players:
  #   print(p.name, "\n")
  
  # deal cards
  dealCards(players, deck)

  game_over = False
  skip = False
  reverse = False
  skipped_turn = False
  reversed_order = False
  # draw target card
  target_card = deck.drawCard()
  discard_pile.append(target_card)
  num_rounds = 0
  num_unos = 0

  # if first card to be flipped over is a wild card, choose random color
  if target_card.action == "Wild Color" or target_card.action == "Wild Draw 4":
    target_card.color = random.choice(["Red", "Blue", "Green", "Yellow"])
    # print("Color changed to", target_card.color)

  while not game_over:
    # print("top of while loop")
    # reverse player list if needed
    if (reverse == True) and (reversed_order == False):
      # copy players list
      new_order = players[current_player_i:]
   
      # add players that went before current list to back of list sequentially
      for player in players[:current_player_i]:
        new_order.append(player)

      # reverse order of list-- current player is now at end
      new_order.reverse()

      # rename to players for consistency
      players = new_order
      reversed_order = True
      reverse = False
      # print("Order Reversed! New order:")
      # for player in players:
      #   print(player.name)

    # each iteration of loop plays 1 player's turn
    for player in players:
      # print("Target Card:")
      # target_card.show()
      # print()
      # print("Current player:", player.name)
      # print("Hand")
      # for c in player.hand:
      #   c.show()

      # integrate discard pile back into deck when low
      if len(deck.cards) <= 16:
        #print("dicard card re-added to deck")
        deck.cards.extend(discard_pile)
        deck.shuffle()

      ### CHECK SPECIAL CARD CONDITIONS
      # case for skip card
      if (target_card.action == "Skip") and (skipped_turn == False):
        skip = True
      # case for draw 2 card
      if (target_card.action == "Draw 2") and (skipped_turn == False):
        skip = True
        # draw 2 cards
        for i in range(2):
          player.draw(deck)
      # case for reverse card
      if (target_card.action == "Reverse") and (reversed_order == False):
        reverse = True
        ### reversing order starting from previous player who played the reverse card
        current_player_plus1 = player
        current_player_i_plus1 = players.index(current_player_plus1)
        # if current player index > 0, previous player is at i-1
        if current_player_i_plus1 > 0:
          current_player_i = current_player_i_plus1 - 1
        # otherwise previous player is last player in players list
        else:
          current_player_i = len(players) - 1
        # break out of loop in order to reorder players
        break
      # case for wild draw card
      if target_card.action == "Wild Draw 4":
        # draw 4 cards
        for i in range(4):
          player.draw(deck)

      # if skip card has been played, continue to next player
      if (skip == True) and (skipped_turn == False):
        # ensure that only one turn is skipped
        skipped_turn = True
        skip = False
        #print(player.name, "turn skipped")
        continue 
    
      #print("This should not run if turn skipped")

      ### PLAYER PLAYS HAND
      prev_target = target_card
      card_played = choose_card(target_card, player, deck)
      if card_played is not None:
        target_card = card_played
        ## add played card to discard pile
        discard_pile.append(card_played)
        ## remove card from player's hand
        player.hand.remove(card_played)
        ## reset skipped_turn and reversed_order when new card played
        skipped_turn = False
        reversed_order = False
        
        # if card played is wild card, change color to color that player has most of
        if card_played.action == "Wild Color" or card_played.action == "Wild Draw 4":
          new_color = player.get_color_preference()
          target_card.color = new_color
          #print("Color Changed to", target_card.color)

      ### CHECK FOR UNO 
      if len(player.hand) == 1:
        num_unos += 1
        #print("Uno!")

      ### CHECK FOR GAME OVER
      if len(player.hand) == 0:
        game_over = True
        winner = player
        if player.name == "p0":
          value = 1
        else:
          value = 0
        break      
    
    ### INCREMENT NUM ROUNDS
    num_rounds +=1
  #num_unos_total.append(num_unos)

  ### END OF GAME
  #print("The winner is {}".format(winner.name))
  #print("Number of Rounds: {}".format(num_rounds))
  #print("Number of Unos: {}".format(num_unos))
  return value
  
        



In [ ]:
def play_n_games(n = 100, numPlayers = 2, strategies = ("random", "random"), special_cards = True):
  #num_unos_total = []
  p1_wins = 0
  for i in range(n):
    value = play_game(numPlayers, strategies, special_cards, seed = i)
    p1_wins += value
  #total_unos = sum(num_unos_total)
  #print("total unos: {}".format(total_unos))
  return p1_wins

In [ ]:
num_rows = 27
data = {"n": [10000 for i in range(num_rows)],
        'group': [1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6],
        "num_players": [2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2],
        "strategies": [("random", "random"), ("color", "random"), ("number", "random"),
                       ("random", "random"), ("color", "random"), ("number", "random"),
                       ("save specials", "random"), ("use specials", "random"), 
                       ("random", "random", "random"), ("color", "random", "random"), ("number", "random", "random"),
                       ("random", "random", "random"), ("color", "random", "random"), ("number", "random", "random"),
                       ("save specials", "random", "random"), ("use specials", "random", "random"), ("color", "number", "save specials"), 
                       ("random", "number", "save specials"), ("random", "random", "save specials"), ("random", "random", "number"),
                       ("use specials", "use specials"), ("number", "use specials"), ("save specials", "use specials"),
                       ("number", "number"), ("number", "save specials"), ("save specials", "save specials"), ("save specials", "number")
                       ],
        "special_cards": [False, False, False, True, True, True, True, True, False, False, False, True, True, True, True, True, True, True, True, True,
                          True, True, True, True, True, True, True]}
df = pd.DataFrame(data)
df.reset_index()

,index,n,group,num_players,strategies,special_cards
0,0,10000,1,2,"(random, random)",False
1,1,10000,1,2,"(color, random)",False
2,2,10000,1,2,"(number, random)",False
3,3,10000,2,2,"(random, random)",True
4,4,10000,2,2,"(color, random)",True
5,5,10000,2,2,"(number, random)",True
6,6,10000,2,2,"(save specials, random)",True
7,7,10000,2,2,"(use specials, random)",True
8,8,10000,3,3,"(random, random, random)",False
9,9,10000,3,3,"(color, random, random)",False


In [ ]:
p0_wins = []
for i in range(num_rows):
  win = play_n_games(n = df["n"][i], numPlayers = df["num_players"][i], strategies = df["strategies"][i], special_cards = df["special_cards"][i])
  p0_wins.append(win)

In [ ]:
p0_wins

[4978,
 5147,
 4876,
 4975,
 5007,
 5101,
 5127,
 5161,
 3401,
 3329,
 3296,
 3353,
 3362,
 3488,
 3504,
 3349,
 3185,
 3241,
 3264,
 3337,
 5036,
 4984,
 5089,
 4990,
 5015,
 5019,
 4957]

In [ ]:
df["p0_wins"] = p0_wins
df

,n,group,num_players,strategies,special_cards,p0_wins
0,10000,1,2,"(random, random)",False,4978
1,10000,1,2,"(color, random)",False,5147
2,10000,1,2,"(number, random)",False,4876
3,10000,2,2,"(random, random)",True,4975
4,10000,2,2,"(color, random)",True,5007
5,10000,2,2,"(number, random)",True,5101
6,10000,2,2,"(save specials, random)",True,5127
7,10000,2,2,"(use specials, random)",True,5161
8,10000,3,3,"(random, random, random)",False,3401
9,10000,3,3,"(color, random, random)",False,3329


In [ ]:
df.to_csv("data2.csv")